In [1]:
import copy
import sqlite3
import numpy as np
import pandas as pd
from rdkit import Chem

RDKit WARNING: [09:51:55] Enabling RDKit 2019.09.3 jupyter extensions
[09:51:55] Enabling RDKit 2019.09.3 jupyter extensions


#### 1. Extracting data from Database

In [2]:
def call_my_query(db_file, my_query):
    ## connect to the SQLIte database
    my_connection = sqlite3.connect(db_file)

    ## create a cursor object
    my_cursor = my_connection.cursor()

    ## excute the query
    my_cursor.execute(my_query)

    ## fetch all the rows
    rows = my_cursor.fetchall()
    
    ## export the results
    data_list = [row for row in rows]

    my_connection.close()
    return data_list

def extract_tables(db_file, table_name):
    ## extract table data from SQLite DB
    my_query_colName = f"PRAGMA table_info({table_name})"
    colName_list = call_my_query(db_file, my_query_colName)

    my_query_data = f"SELECT * FROM {table_name}"
    data_list = call_my_query(db_file, my_query_data)

    ## clean up data
    dataDict = {}
    for row_tuple in data_list:
        idx = row_tuple[0]
        dataDict[idx] = {}

        for col in colName_list:
            colIdx, colName = col[0], col[1]
            dataDict[idx][colName] = row_tuple[colIdx]
    return dataDict

In [3]:
db_file = './results/Compounds_All.mmpdb'
dataDict_tables = {}

for table_name in ["pair", "compound", "compound_property", "property_name", "constant_smiles",
                   "rule", "rule_smiles", "rule_environment", "rule_environment_statistics", "environment_fingerprint"]:
    dataDict_table = extract_tables(db_file, table_name)
    dataDict_tables[table_name] = pd.DataFrame.from_dict(dataDict_table).T
    # print(table_name)

#### 2. clean up data

In [4]:
dataTable = copy.deepcopy(dataDict_tables["pair"])
dataTable.head(3)

## ------------------- add compound structure & property data -------------------
table_merge = dataDict_tables["compound"]

## compound-1 (from)
dataTable = dataTable.merge(table_merge, left_on=['compound1_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'clean_smiles', 'clean_num_heavies'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'public_id':'KT_number_1', 'input_smiles':'smiles_1'}, inplace=True)

## compound-2 (to)
dataTable = dataTable.merge(table_merge, left_on=['compound2_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'clean_smiles', 'clean_num_heavies'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'public_id':'KT_number_2', 'input_smiles':'smiles_2'}, inplace=True)

## ------------------- add compound prop data -------------------
table_merge = dataDict_tables["compound_property"]

## compound-1 (from)
dataTable = dataTable.merge(table_merge, left_on=['compound1_id'], right_on=['compound_id'])
dataTable.drop(columns=['id_y', 'compound_id'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'value':'property_values_1'}, inplace=True)

## compound-2 (to)
dataTable = dataTable.merge(table_merge, left_on=['compound2_id', 'property_name_id'], right_on=['compound_id', 'property_name_id'])
dataTable.drop(columns=['id_y', 'compound_id'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'value':'property_values_2'}, inplace=True)

## add property name
table_merge = dataDict_tables["property_name"]
dataTable = dataTable.merge(table_merge, left_on=['property_name_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'name':'property_name'}, inplace=True)

## ------------------- add constant pieces data of the match pair -------------------
table_merge = dataDict_tables["constant_smiles"]
dataTable = dataTable.merge(table_merge, left_on=['constant_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'smiles':'constant_smiles'}, inplace=True)

## ------------------- add rule env data -------------------
table_merge = dataDict_tables["rule_environment"]
dataTable = dataTable.merge(table_merge, left_on=['rule_environment_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'radius':'rule_env_radius', 'num_pairs':'rule_env_num_pairs'}, inplace=True)

## ------------------- add rule info -------------------
table_merge = dataDict_tables["rule"]
dataTable = dataTable.merge(table_merge, left_on=['rule_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)    #'rule_id'
dataTable.rename(columns={'id_x':'id'}, inplace=True)

table_merge = dataDict_tables["rule_smiles"]
dataTable = dataTable.merge(table_merge, left_on=['from_smiles_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'from_smiles_id', 'num_heavies'], inplace=True)    #'num_heavies'
dataTable.rename(columns={'id_x':'id', 'smiles':'rule_from_smiles'}, inplace=True)

table_merge = dataDict_tables["rule_smiles"]
dataTable = dataTable.merge(table_merge, left_on=['to_smiles_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'to_smiles_id', 'num_heavies'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'smiles':'rule_to_smiles'}, inplace=True)

## ------------------- add rule env stats -------------------
table_merge = dataDict_tables["rule_environment_statistics"]
dataTable = dataTable.merge(table_merge, 
                            left_on=['rule_environment_id', 'property_name_id'], 
                            right_on=['rule_environment_id', 'property_name_id'])

drop_cols = ['kurtosis', 'skewness', 'paired_t', 'p_value', 'q1', 'q3', 'median', 'std']
dataTable.drop(columns=['id_y']+drop_cols, inplace=True)
dataTable.rename(columns={'id_x':'id', 'count':'rule_env_count', 'avg':'rule_env_avg', 
                          'min':'rule_env_min', 'max':'rule_env_max'}, inplace=True)

## ------------------- add rule env environment_fingerprint data -------------------
table_merge = dataDict_tables["environment_fingerprint"]
## to be added

## ------------------- remove useless cols -------------------
dataTable.drop(columns=['id', 'compound1_id', 'compound2_id', 'constant_id', 'rule_environment_id', 'property_name_id'], inplace=True)
print(dataTable.shape)
dataTable.head(3)

(283800, 18)


,KT_number_1,smiles_1,KT_number_2,smiles_2,property_values_1,property_values_2,property_name,constant_smiles,rule_id,environment_fingerprint_id,rule_env_radius,rule_env_num_pairs,rule_from_smiles,rule_to_smiles,rule_env_count,rule_env_avg,rule_env_min,rule_env_max
0,KT-0010336,n1(c3c(c(c1C#CCNC(=O)OC(C)(C)C)/C=C/2\CC(OC2=O...,KT-0007377,n1(c3c(c(c1)/C=C/2\CC(OC2=O)(COC(=O)C(C)(C)C)C...,11.614233,6.871179,permeability,[*:1]/C=C1/CC(CO)(COC(=O)C(C)(C)C)OC1=O.[*:2]C...,1,1,0,1,[*:1]c1c([*:2])n(C)c2ccccc12,[*:2]c1ccc2c(c1)c([*:1])cn2C,1.0,-4.743054,-4.743054,-4.743054
1,KT-0010336,n1(c3c(c(c1C#CCNC(=O)OC(C)(C)C)/C=C/2\CC(OC2=O...,KT-0007377,n1(c3c(c(c1)/C=C/2\CC(OC2=O)(COC(=O)C(C)(C)C)C...,11.614233,6.871179,permeability,[*:1]/C=C1/CC(CO)(COC(=O)C(C)(C)C)OC1=O.[*:2]C...,1,2,1,1,[*:1]c1c([*:2])n(C)c2ccccc12,[*:2]c1ccc2c(c1)c([*:1])cn2C,1.0,-4.743054,-4.743054,-4.743054
2,KT-0010336,n1(c3c(c(c1C#CCNC(=O)OC(C)(C)C)/C=C/2\CC(OC2=O...,KT-0007377,n1(c3c(c(c1)/C=C/2\CC(OC2=O)(COC(=O)C(C)(C)C)C...,11.614233,6.871179,permeability,[*:1]/C=C1/CC(CO)(COC(=O)C(C)(C)C)OC1=O.[*:2]C...,1,3,2,1,[*:1]c1c([*:2])n(C)c2ccccc12,[*:2]c1ccc2c(c1)c([*:1])cn2C,1.0,-4.743054,-4.743054,-4.743054


In [5]:
def GeneratePairID(row, col_mol_id_1='KT_number_1', col_mol_id_2='KT_number_2'):
    mol_id_1 = row[col_mol_id_1]
    mol_id_2 = row[col_mol_id_2]
    pair_id = str(mol_id_1) + '=>' + str(mol_id_2)
    
    mol_id_1_num = int(str(mol_id_1).split('-')[1])
    mol_id_2_num = int(str(mol_id_2).split('-')[1])
    pair_couple = (np.min([mol_id_1_num, mol_id_2_num]), np.max([mol_id_1_num, mol_id_2_num]))
    return pd.Series([pair_id, pair_couple])

dataTable[['Pair_id', 'PairInfo']] = dataTable.apply(lambda row: GeneratePairID(row, col_mol_id_1='KT_number_1', col_mol_id_2='KT_number_2'), axis=1)
print(dataTable.shape)

################################################################################################
def calculate_heavy_atoms(molecule_smiles):
    try:
        mol = Chem.MolFromSmiles(molecule_smiles)
        num_heavy_atoms = mol.GetNumHeavyAtoms()
    except Exception as e:
        print('Error', e)
        num_heavy_atoms = np.nan
    return num_heavy_atoms

dataTable['constant_size'] = dataTable['constant_smiles'].apply(calculate_heavy_atoms)
dataTable.sort_values(by=['PairInfo', 'Pair_id', 'rule_env_radius', 'constant_size'], ascending=[True, True, True, False], inplace=True)
print(dataTable.shape)

################################################################################################

(283800, 20)
(283800, 21)


In [6]:
def calculate_heavy_atoms(molecule_smiles):
    try:
        mol = Chem.MolFromSmiles(molecule_smiles)
        num_heavy_atoms = mol.GetNumHeavyAtoms()
    except Exception as e:
        print('Error', e)
        num_heavy_atoms = np.nan
    return num_heavy_atoms

dataTable['constant_size'] = dataTable['constant_smiles'].apply(calculate_heavy_atoms)
dataTable.sort_values(by=['PairInfo', 'Pair_id', 'rule_env_radius', 'constant_size'], ascending=[True, True, True, False], inplace=True)
dataTable.to_csv(f'./results/Compounds_All_4_informatics.csv', index=False)
dataTable.head(3)

,KT_number_1,smiles_1,KT_number_2,smiles_2,property_values_1,property_values_2,property_name,constant_smiles,rule_id,environment_fingerprint_id,...,rule_env_num_pairs,rule_from_smiles,rule_to_smiles,rule_env_count,rule_env_avg,rule_env_min,rule_env_max,Pair_id,PairInfo,constant_size
2766,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,18.667024,3.952992,permeability,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,207,[*:1]C,[*:1][H],110.0,-0.436933,-14.714033,12.530116,KT-0000377=>KT-0000559,"(377, 559)",18
2825,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,1.289292,1.336693,efflux,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,207,[*:1]C,[*:1][H],64.0,-3.446388,-157.859995,60.373535,KT-0000377=>KT-0000559,"(377, 559)",18
3961,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,18.667024,3.952992,permeability,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,4988,...,19,[*:1]C,[*:1][H],4.0,-5.274685,-14.714033,0.685420,KT-0000377=>KT-0000559,"(377, 559)",18


#### 3. removed the "duplicated" rows

In [7]:
dataTable.sort_values(by=['PairInfo', 'rule_env_radius', 'constant_size'], ascending=[True, True, False], inplace=True)
dataTable_rmDup = dataTable.drop_duplicates(subset=['PairInfo', 'property_name'], keep='first', inplace=False)
print(dataTable_rmDup.shape)
dataTable_rmDup.head(3)

(18588, 21)


,KT_number_1,smiles_1,KT_number_2,smiles_2,property_values_1,property_values_2,property_name,constant_smiles,rule_id,environment_fingerprint_id,...,rule_env_num_pairs,rule_from_smiles,rule_to_smiles,rule_env_count,rule_env_avg,rule_env_min,rule_env_max,Pair_id,PairInfo,constant_size
2766,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,18.667024,3.952992,permeability,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,207,[*:1]C,[*:1][H],110.0,-0.436933,-14.714033,12.530116,KT-0000377=>KT-0000559,"(377, 559)",18
2825,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,1.289292,1.336693,efflux,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,207,[*:1]C,[*:1][H],64.0,-3.446388,-157.859995,60.373535,KT-0000377=>KT-0000559,"(377, 559)",18
12738,KT-0001321,N2(c1c(cc(cc1)CNC(=O)C)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,1.109992,18.667024,permeability,[*:1]c1ccc2c(c1)n(C)c(=O)n2C1CCC(=O)NC1=O,16903,78,...,1,[*:1]CNC(C)=O,[*:1][H],1.0,17.557032,17.557032,17.557032,KT-0001321=>KT-0000377,"(377, 1321)",19


#### append symetric rows

In [8]:
rename_symetric_dict = {
    'KT_number_1': 'KT_number_2',
    'smiles_1': 'smiles_2',
    'KT_number_2': 'KT_number_1',
    'smiles_2': 'smiles_1',
    'property_values_1': 'property_values_2',
    'property_values_2': 'property_values_1', 
    'rule_from_smiles': 'rule_to_smiles',
    'rule_to_smiles': 'rule_from_smiles'}
dataTable_rmDup_symetric = dataTable_rmDup.rename(columns=rename_symetric_dict, inplace=False)
dataTable_rmDup_symetric['Pair_id'] = dataTable_rmDup_symetric['KT_number_1'] + '=>' + dataTable_rmDup_symetric['KT_number_2']
for col in ['rule_env_avg', 'rule_env_min', 'rule_env_max']:
    dataTable_rmDup_symetric[col] = dataTable_rmDup_symetric[col] * -1
dataTable_rmDup_symetric.head(3)

,KT_number_2,smiles_2,KT_number_1,smiles_1,property_values_2,property_values_1,property_name,constant_smiles,rule_id,environment_fingerprint_id,...,rule_env_num_pairs,rule_to_smiles,rule_from_smiles,rule_env_count,rule_env_avg,rule_env_min,rule_env_max,Pair_id,PairInfo,constant_size
2766,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,18.667024,3.952992,permeability,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,207,[*:1]C,[*:1][H],110.0,0.436933,14.714033,-12.530116,KT-0000559=>KT-0000377,"(377, 559)",18
2825,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,1.289292,1.336693,efflux,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,207,[*:1]C,[*:1][H],64.0,3.446388,157.859995,-60.373535,KT-0000559=>KT-0000377,"(377, 559)",18
12738,KT-0001321,N2(c1c(cc(cc1)CNC(=O)C)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,1.109992,18.667024,permeability,[*:1]c1ccc2c(c1)n(C)c(=O)n2C1CCC(=O)NC1=O,16903,78,...,1,[*:1]CNC(C)=O,[*:1][H],1.0,-17.557032,-17.557032,-17.557032,KT-0000377=>KT-0001321,"(377, 1321)",19


In [9]:
dataTable_rmDup_all = dataTable_rmDup._append(dataTable_rmDup_symetric, ignore_index=True)
dataTable_rmDup_all['rule_env_min'] = dataTable_rmDup_all['rule_env_min'].apply(lambda x:round(x, 2)).astype('str')
dataTable_rmDup_all['rule_env_max'] = dataTable_rmDup_all['rule_env_max'].apply(lambda x:round(x, 2)).astype('str')
dataTable_rmDup_all['rule_env_range'] = '('+ dataTable_rmDup_all['rule_env_min'] + ',' + dataTable_rmDup_all['rule_env_max'] +')'

In [10]:
dataTable_rmDup_all.sort_values(by=['PairInfo', 'property_name', 'Pair_id'], ascending=[True, True, True], inplace=True)
dataTable_rmDup_all = dataTable_rmDup_all.reset_index(drop=True)
dataTable_rmDup_all.to_csv(f'./results/Compounds_All_4_informatics_rmDups.csv', index=False)
dataTable_rmDup_all

,KT_number_1,smiles_1,KT_number_2,smiles_2,property_values_1,property_values_2,property_name,constant_smiles,rule_id,environment_fingerprint_id,...,rule_from_smiles,rule_to_smiles,rule_env_count,rule_env_avg,rule_env_min,rule_env_max,Pair_id,PairInfo,constant_size,rule_env_range
0,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,1.289292,1.336693,efflux,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,[*:1]C,[*:1][H],64.0,-3.446388,-157.86,60.37,KT-0000377=>KT-0000559,"(377, 559)",18,"(-157.86,60.37)"
1,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,1.336693,1.289292,efflux,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,[*:1][H],[*:1]C,64.0,3.446388,157.86,-60.37,KT-0000559=>KT-0000377,"(377, 559)",18,"(157.86,-60.37)"
2,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,18.667024,3.952992,permeability,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,[*:1]C,[*:1][H],110.0,-0.436933,-14.71,12.53,KT-0000377=>KT-0000559,"(377, 559)",18,"(-14.71,12.53)"
3,KT-0000559,N2(c1c(cccc1)NC2=O)C3CCC(=O)NC3=O,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,3.952992,18.667024,permeability,[*:1]n1c(=O)n(C2CCC(=O)NC2=O)c2ccccc21,3050,78,...,[*:1][H],[*:1]C,110.0,0.436933,14.71,-12.53,KT-0000559=>KT-0000377,"(377, 559)",18,"(14.71,-12.53)"
4,KT-0000377,N2(c1c(cccc1)N(C2=O)C)C3CCC(=O)NC3=O,KT-0001321,N2(c1c(cc(cc1)CNC(=O)C)N(C2=O)C)C3CCC(=O)NC3=O,18.667024,1.109992,permeability,[*:1]c1ccc2c(c1)n(C)c(=O)n2C1CCC(=O)NC1=O,16903,78,...,[*:1][H],[*:1]CNC(C)=O,1.0,-17.557032,-17.56,-17.56,KT-0000377=>KT-0001321,"(377, 1321)",19,"(-17.56,-17.56)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37171,KT-0199164,Fc3c1c(n(nc1C(=O)Nc2cc(ccc2)C(=O)N)C)ccc3-c4cc...,KT-0199156,Fc3c1c(n(nc1C(=O)Nc2cc(ccc2)C(=O)N)C)ccc3-c4cc...,2.400000,0.303000,F%_Rat,[*:1]c1nn(C)c(=O)n1-c1ccc(-c2ccc3c(c(C(=O)Nc4c...,3050,78,...,[*:1]C,[*:1][H],91.0,-2.874227,-27.33,21.68,KT-0199164=>KT-0199156,"(199156, 199164)",36,"(-27.33,21.68)"
37172,KT-0199167,N1(N=CN(C1=O)c2ccc(cc2)-c3cc(c(cc3)C)N4CCN(C4=...,KT-0199168,Fc1c(cccc1-c2ccc(cc2)N3C=NN(C3=O)C)N4CCN(C4=O)...,0.305652,0.011823,EstFa_Rat,[*:2]N1CCN(c2cccc(C(N)=O)c2)C1=O.[*:1]c1ccc(-n...,13320,1,...,[*:1]c1ccc(C)c([*:2])c1,[*:1]c1cccc([*:2])c1F,1.0,-0.293829,-0.29,-0.29,KT-0199167=>KT-0199168,"(199167, 199168)",28,"(-0.29,-0.29)"
37173,KT-0199168,Fc1c(cccc1-c2ccc(cc2)N3C=NN(C3=O)C)N4CCN(C4=O)...,KT-0199167,N1(N=CN(C1=O)c2ccc(cc2)-c3cc(c(cc3)C)N4CCN(C4=...,0.011823,0.305652,EstFa_Rat,[*:2]N1CCN(c2cccc(C(N)=O)c2)C1=O.[*:1]c1ccc(-n...,13320,1,...,[*:1]c1cccc([*:2])c1F,[*:1]c1ccc(C)c([*:2])c1,1.0,0.293829,0.29,0.29,KT-0199168=>KT-0199167,"(199167, 199168)",28,"(0.29,0.29)"
37174,KT-0199167,N1(N=CN(C1=O)c2ccc(cc2)-c3cc(c(cc3)C)N4CCN(C4=...,KT-0199168,Fc1c(cccc1-c2ccc(cc2)N3C=NN(C3=O)C)N4CCN(C4=O)...,27.400000,1.080000,F%_Rat,[*:2]N1CCN(c2cccc(C(N)=O)c2)C1=O.[*:1]c1ccc(-n...,13320,1,...,[*:1]c1ccc(C)c([*:2])c1,[*:1]c1cccc([*:2])c1F,1.0,-26.320000,-26.32,-26.32,KT-0199167=>KT-0199168,"(199167, 199168)",28,"(-26.32,-26.32)"
